In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist
from scipy.stats import qmc, lognorm, gumbel_r, uniform

In [ ]:


# -----------------------------
# CONFIG
# -----------------------------
n = 200      # number of design points to be generated
d = 7        # number of variables
tries = 50   # number of LHS samples to evaluate

# -----------------------------
# INPUT DISTRIBUTIONS
# -----------------------------

# Lognormal: mean, std → shape (sigma), scale (exp(mu))
def get_lognormal_params(mean, std):
    sigma = np.log(np.sqrt(1 + (std / mean)**2))   
    mu = np.log(mean) - 0.5 * sigma**2    # ???           
    return sigma, np.exp(mu)              

# Gumbel: mean, std → loc, scale
def get_gumbel_params(mean, std):
    beta = std * np.sqrt(6) / np.pi
    loc = mean - 0.5772 * beta
    return loc, beta

# Uniform: mean ± sqrt(3)*std
def get_uniform_bounds(mean, std):
    delta = np.sqrt(3) * std
    return mean - delta, mean + delta


# Column order
col_order = ['sigma_mem_y', 'f_mem', 'sigma_mem', 'E_mem', 'nu_mem', 'sigma_edg', 'sigma_sup']

# -----------------------------
# Define Lognormal Parameters
# -----------------------------
lognorm_vars = {
    'sigma_mem_y': get_lognormal_params(11000, 1650),
    'sigma_mem': get_lognormal_params(4000, 800),
    'E_mem': get_lognormal_params(600000, 90000),
    'sigma_edg': get_lognormal_params(353677.6513, 70735.53026),
    'sigma_sup': get_lognormal_params(400834.6715, 80166.9343),
}

# Gumbel: Mean and std for f_mem
gumbel_mean, gumbel_std = 0.4, 0.12
beta = gumbel_std * np.sqrt(6) / np.pi
loc_gumbel = gumbel_mean - 0.5772 * beta

# Uniform: Mean and std for nu_mem
nu_mean, nu_std = 0.4, 0.0115
a_uni = nu_mean - np.sqrt(3) * nu_std
b_uni = nu_mean + np.sqrt(3) * nu_std

# -----------------------------
# LHS + MAXIMIN SEARCH (50 tries)
# -----------------------------

def lhs_maximin(n, d, tries=50):
    best_sample = None
    best_score = -np.inf
    
    for _ in range(tries):
        # Generate LHS sample
        sampler = qmc.LatinHypercube(d)
        sample = sampler.random(n)
        
        # Calculate pairwise distances to ensure Maximin criterion
        dist = pdist(sample)
        min_dist = dist.min()  # Find the smallest distance between points
        
        # Select the sample with the largest minimum distance (Maximin)
        if min_dist > best_score:
            best_score = min_dist
            best_sample = sample
    
    return best_sample

# Generate 200 points using LHS and Maximin criterion
X_prob = lhs_maximin(n, d, tries)

# -----------------------------
# INVERSE TRANSFORM TO PHYSICAL VALUES
# -----------------------------
X_phys = pd.DataFrame(columns=col_order)

# Applying transformations based on the new formulas
X_phys['sigma_mem_y'] = lognorm.ppf(X_prob[:, 0], s=lognorm_vars['sigma_mem_y'][0], scale=lognorm_vars['sigma_mem_y'][1])
X_phys['f_mem'] = gumbel_r.ppf(X_prob[:, 1], loc=loc_gumbel, scale=beta)
X_phys['sigma_mem'] = lognorm.ppf(X_prob[:, 2], s=lognorm_vars['sigma_mem'][0], scale=lognorm_vars['sigma_mem'][1])
X_phys['E_mem'] = lognorm.ppf(X_prob[:, 3], s=lognorm_vars['E_mem'][0], scale=lognorm_vars['E_mem'][1])
X_phys['nu_mem'] = uniform.ppf(X_prob[:, 4], loc=a_uni, scale=b_uni - a_uni)
X_phys['sigma_edg'] = lognorm.ppf(X_prob[:, 5], s=lognorm_vars['sigma_edg'][0], scale=lognorm_vars['sigma_edg'][1])
X_phys['sigma_sup'] = lognorm.ppf(X_prob[:, 6], s=lognorm_vars['sigma_sup'][0], scale=lognorm_vars['sigma_sup'][1])

# -----------------------------
# SAVE OR VIEW
# -----------------------------
X_phys = X_phys.round(4)  # Rounding the values for clean output
print(X_phys.head())  # Print the first 5 rows for preview

# Save the final 200-point design to a CSV file
X_phys.to_csv("lhs_maximin_sunsail_200.csv", index=False)


   sigma_mem_y   f_mem  sigma_mem        E_mem  nu_mem    sigma_edg  \
0   11158.6749  0.4993  4064.2180  603769.8678  0.4008  345517.5134   
1   11166.8784  0.3790  4003.2122  603990.8371  0.4053  352451.9070   
2   11163.9911  0.2786  3942.0655  603087.1636  0.4166  352326.8325   
3   11058.9539  0.3582  3969.4459  600136.5585  0.3895  340673.8676   
4   10890.7440  0.5126  4064.5295  595326.0254  0.4000  351213.1727   

     sigma_sup  
0  415959.9846  
1  417161.2418  
2  413136.7560  
3  394726.3072  
4  402945.0846  
